## Model Developmemt and Performance Analysis

### Methodology

Number of classes are selected and data is filtered from the original database. After cleaning the description, various text feature extraction methods are applied and finally various classifiers are trained on training split and the results are tested on the testing split of data.



For evaluation we will consider, F1 score as the metrices as our data is imbalanced. Therefore we need to consider both precision and recall and as all classes are equally likely we would prefer F1-score as a evaluation metrices

### Data Selection and Description Cleaning

In [ ]:
import pandas as pd 
import numpy as np 
import string
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Intermidiate.csv")
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,year,month,discount%,Main_Category
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23+00:00,alisha-solid-women-s-cycling-shorts,Alisha Solid Women's Cycling Shorts,"['[""Clothing ', "" Women's Clothing "", ' Linger...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,0.0,0.0,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",2016,3,62.06,Clothing
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23+00:00,fabhomedecor-fabric-double-sofa-bed,FabHomeDecor Fabric Double Sofa Bed,"['[""Furniture ', ' Living Room Furniture ', ' ...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,0.0,0.0,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",2016,3,29.58,Furniture
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23+00:00,aw-bellies,AW Bellies,"['[""Footwear ', "" Women's Footwear "", ' Baller...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,0.0,0.0,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",2016,3,50.05,Footwear
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23+00:00,alisha-solid-women-s-cycling-shorts,Alisha Solid Women's Cycling Shorts,"['[""Clothing ', "" Women's Clothing "", ' Linger...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,0.0,0.0,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",2016,3,61.80,Clothing
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23+00:00,sicons-all-purpose-arnica-dog-shampoo,Sicons All Purpose Arnica Dog Shampoo,"['[""Pet Supplies ', ' Grooming ', ' Skin & Coa...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,0.0,0.0,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",2016,3,4.55,Pet Supplies


In [ ]:
df = data[["description","Main_Category"]]
df.head()

,description,Main_Category
0,Key Features of Alisha Solid Women's Cycling S...,Clothing
1,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,Furniture
2,Key Features of AW Bellies Sandals Wedges Heel...,Footwear
3,Key Features of Alisha Solid Women's Cycling S...,Clothing
4,Specifications of Sicons All Purpose Arnica Do...,Pet Supplies


In [ ]:
class_dist = df["Main_Category"].value_counts()
classes = class_dist[class_dist>9]

In [ ]:
classes

Clothing                             6198
Jewellery                            3531
Footwear                             1227
Mobiles & Accessories                1099
Automotive                           1012
Home Decor & Festive Needs            929
Beauty and Personal Care              710
Home Furnishing                       700
Kitchen & Dining                      647
Computers                             578
Watches                               530
Baby Care                             483
Tools & Hardware                      391
Toys & School Supplies                330
Pens & Stationery                     313
Bags, Wallets & Belts                 265
Furniture                             180
Sports & Fitness                      166
Cameras & Accessories                  82
Home Improvement                       81
Health & Personal Care Appliances      43
Sunglasses                             35
Gaming                                 35
Pet Supplies                      

In [ ]:
df = df[df["Main_Category"].isin(classes.index.values)]

In [ ]:
df.shape

(19663, 2)

In [ ]:
puntuations = string.punctuation
stop_words = nltk.corpus.stopwords.words('english')
custom_list = []
filter_list = stop_words + custom_list

In [ ]:
def preprocess(text):
    text = str(text)
    text = text.lower()
    c_text = ""
    for char in list(text):
        if char not in puntuations:
            c_text+=char
        elif char in [",","-"]:
            c_text+=" "
    text = c_text
    text = "".join(eword+" " for eword in c_text.split() if eword.isalpha() and eword not in filter_list)
    return text.strip()

In [ ]:
preprocess(df.description[0])

'key features alisha solid womens cycling shorts cotton lycra navy red navy specifications alisha solid womens cycling shorts shorts details number contents sales package pack fabric cotton lycra type cycling shorts general details pattern solid ideal womens fabric care gentle machine wash lukewarm water bleach additional details style code box shorts'

In [ ]:
feature_text = df.description.apply(preprocess)

## Label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
label = le.fit_transform(df["Main_Category"])

In [ ]:
max(label)

27

## Feature extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tfidf = TfidfVectorizer()
cv = CountVectorizer()

In [ ]:
feature_cv = cv.fit_transform(feature_text)
feature_tfidf = tfidf.fit_transform(feature_text)

## Model Training

### Statisitcal algorithms

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(feature_cv, label, test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(feature_tfidf, label, test_size=0.2, random_state=42)

In [ ]:
clf1 = MultinomialNB()
clf2 = SVC()
clf3 = RandomForestClassifier(random_state=42)
clf4 = LogisticRegression(random_state=42)
clf5 = XGBClassifier()
clf6 = MLPClassifier()

#### For CV

In [ ]:
clf1.fit(X_train_cv,y_train_cv)
y_test_pred_cv = clf1.predict(X_test_cv)
print(classification_report(y_test_cv,y_test_pred_cv))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       181
           1       0.85      0.55      0.67       110
           2       0.79      0.58      0.67        38
           3       0.82      0.96      0.88       147
           4       1.00      0.25      0.40        20
           5       0.97      0.98      0.98      1207
           6       0.86      0.79      0.83       131
           8       0.97      0.98      0.98       259
           9       0.92      0.95      0.94        38
          10       0.83      0.83      0.83         6
          11       1.00      0.55      0.71        11
          12       1.00      0.40      0.57         5
          13       0.91      0.97      0.94       187
          14       0.00      0.00      0.00         3
          15       0.90      1.00      0.95       143
          16       1.00      0.21      0.35        19
          17       0.90      1.00      0.95       690
          18       0.97    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf2.fit(X_train_cv,y_train_cv)
y_test_pred_cv = clf2.predict(X_test_cv)
print(classification_report(y_test_cv,y_test_pred_cv))

              precision    recall  f1-score   support

           0       0.99      0.94      0.97       181
           1       0.95      0.75      0.84       110
           2       0.91      0.82      0.86        38
           3       0.98      0.93      0.95       147
           4       1.00      0.75      0.86        20
           5       0.91      1.00      0.95      1207
           6       0.90      0.92      0.91       131
           8       0.99      0.96      0.98       259
           9       1.00      0.92      0.96        38
          10       1.00      0.50      0.67         6
          11       1.00      0.36      0.53        11
          12       1.00      0.20      0.33         5
          13       0.87      0.95      0.91       187
          14       0.00      0.00      0.00         3
          15       0.90      0.92      0.91       143
          16       1.00      0.32      0.48        19
          17       0.99      0.99      0.99       690
          18       0.88    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf3.fit(X_train_cv,y_train_cv)
y_test_pred_cv = clf3.predict(X_test_cv)
print(classification_report(y_test_cv,y_test_pred_cv))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       181
           1       0.99      0.69      0.81       110
           2       0.97      0.87      0.92        38
           3       0.99      0.97      0.98       147
           4       1.00      0.90      0.95        20
           5       0.97      1.00      0.98      1207
           6       0.97      0.95      0.96       131
           8       0.99      0.98      0.98       259
           9       0.95      0.97      0.96        38
          10       1.00      0.83      0.91         6
          11       1.00      0.91      0.95        11
          12       1.00      0.40      0.57         5
          13       0.91      0.98      0.94       187
          14       1.00      0.33      0.50         3
          15       0.95      0.97      0.96       143
          16       1.00      0.53      0.69        19
          17       0.99      1.00      0.99       690
          18       0.93    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf4.fit(X_train_cv,y_train_cv)
y_test_pred_cv = clf4.predict(X_test_cv)
print(classification_report(y_test_cv,y_test_pred_cv))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       181
           1       0.91      0.83      0.87       110
           2       0.90      0.95      0.92        38
           3       0.95      0.99      0.97       147
           4       0.95      0.95      0.95        20
           5       0.99      1.00      0.99      1207
           6       0.95      0.95      0.95       131
           8       1.00      0.99      1.00       259
           9       1.00      0.95      0.97        38
          10       1.00      0.83      0.91         6
          11       1.00      0.91      0.95        11
          12       0.67      0.40      0.50         5
          13       0.97      0.97      0.97       187
          14       0.00      0.00      0.00         3
          15       0.94      0.98      0.96       143
          16       0.93      0.74      0.82        19
          17       1.00      0.99      0.99       690
          18       0.93    

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf5.fit(X_train_cv,y_train_cv)
y_test_pred_cv = clf5.predict(X_test_cv)
print(classification_report(y_test_cv,y_test_pred_cv))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       181
           1       0.91      0.82      0.86       110
           2       0.92      0.92      0.92        38
           3       0.96      0.93      0.94       147
           4       1.00      0.95      0.97        20
           5       0.99      0.99      0.99      1207
           6       0.98      0.87      0.92       131
           7       0.00      0.00      0.00         0
           8       0.98      0.98      0.98       259
           9       0.95      0.97      0.96        38
          10       0.83      0.83      0.83         6
          11       1.00      0.82      0.90        11
          12       1.00      0.20      0.33         5
          13       0.93      0.94      0.93       187
          14       0.50      0.33      0.40         3
          15       0.90      0.99      0.94       143
          16       1.00      0.63      0.77        19
          17       0.97    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf6.fit(X_train_cv,y_train_cv)
y_test_pred_cv = clf6.predict(X_test_cv)
print(classification_report(y_test_cv,y_test_pred_cv))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       181
           1       0.92      0.84      0.88       110
           2       0.90      0.97      0.94        38
           3       0.97      0.97      0.97       147
           4       1.00      0.95      0.97        20
           5       0.99      1.00      0.99      1207
           6       0.98      0.94      0.96       131
           8       0.99      0.99      0.99       259
           9       1.00      0.95      0.97        38
          10       1.00      0.67      0.80         6
          11       0.92      1.00      0.96        11
          12       0.67      0.40      0.50         5
          13       0.94      0.98      0.96       187
          14       0.67      0.67      0.67         3
          15       0.95      1.00      0.97       143
          16       0.93      0.74      0.82        19
          17       1.00      1.00      1.00       690
          18       0.98    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### For TfIdf

In [ ]:
clf1.fit(X_train_tfidf,y_train_tfidf)
y_test_pred_tfidf = clf1.predict(X_test_tfidf)
print(classification_report(y_test_tfidf,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92       181
           1       1.00      0.41      0.58       110
           2       1.00      0.13      0.23        38
           3       0.98      0.62      0.76       147
           4       1.00      0.05      0.10        20
           5       0.86      1.00      0.92      1207
           6       1.00      0.73      0.85       131
           8       1.00      0.93      0.96       259
           9       1.00      0.66      0.79        38
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00        11
          12       0.00      0.00      0.00         5
          13       0.91      0.91      0.91       187
          14       0.00      0.00      0.00         3
          15       0.94      0.80      0.86       143
          16       0.00      0.00      0.00        19
          17       0.71      1.00      0.83       690
          18       0.98    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf2.fit(X_train_tfidf,y_train_tfidf)
y_test_pred_tfidf = clf2.predict(X_test_tfidf)
print(classification_report(y_test_tfidf,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       181
           1       0.95      0.81      0.87       110
           2       0.89      0.89      0.89        38
           3       0.99      0.96      0.98       147
           4       1.00      0.90      0.95        20
           5       0.95      1.00      0.97      1207
           6       0.98      0.92      0.95       131
           8       0.99      0.99      0.99       259
           9       0.95      0.95      0.95        38
          10       1.00      0.67      0.80         6
          11       1.00      0.91      0.95        11
          12       1.00      0.20      0.33         5
          13       0.93      0.98      0.96       187
          14       0.00      0.00      0.00         3
          15       0.95      0.99      0.97       143
          16       1.00      0.32      0.48        19
          17       0.99      1.00      0.99       690
          18       0.94    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf3.fit(X_train_tfidf,y_train_tfidf)
y_test_pred_tfidf = clf3.predict(X_test_tfidf)
print(classification_report(y_test_tfidf,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       181
           1       0.99      0.66      0.79       110
           2       1.00      0.87      0.93        38
           3       0.97      0.97      0.97       147
           4       1.00      0.90      0.95        20
           5       0.96      1.00      0.98      1207
           6       0.98      0.92      0.95       131
           8       0.99      0.98      0.98       259
           9       0.95      0.97      0.96        38
          10       1.00      0.83      0.91         6
          11       1.00      0.91      0.95        11
          12       1.00      0.40      0.57         5
          13       0.90      0.96      0.93       187
          14       1.00      0.33      0.50         3
          15       0.93      0.97      0.95       143
          16       1.00      0.37      0.54        19
          17       0.98      1.00      0.99       690
          18       0.91    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf4.fit(X_train_tfidf,y_train_tfidf)
y_test_pred_tfidf = clf4.predict(X_test_tfidf)
print(classification_report(y_test_tfidf,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       181
           1       0.92      0.71      0.80       110
           2       0.89      0.84      0.86        38
           3       0.95      0.95      0.95       147
           4       1.00      0.80      0.89        20
           5       0.95      1.00      0.97      1207
           6       0.94      0.92      0.93       131
           8       0.99      0.98      0.99       259
           9       0.95      0.95      0.95        38
          10       1.00      0.67      0.80         6
          11       1.00      0.73      0.84        11
          12       0.00      0.00      0.00         5
          13       0.92      0.96      0.94       187
          14       0.00      0.00      0.00         3
          15       0.93      0.98      0.96       143
          16       1.00      0.26      0.42        19
          17       0.98      1.00      0.99       690
          18       0.92    

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf5.fit(X_train_tfidf,y_train_tfidf)
y_test_pred_tfidf = clf5.predict(X_test_tfidf)
print(classification_report(y_test_tfidf,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       181
           1       0.88      0.82      0.85       110
           2       0.90      0.95      0.92        38
           3       0.96      0.95      0.96       147
           4       1.00      0.90      0.95        20
           5       0.98      0.99      0.99      1207
           6       0.95      0.93      0.94       131
           8       0.99      0.97      0.98       259
           9       0.95      0.97      0.96        38
          10       1.00      0.83      0.91         6
          11       1.00      0.82      0.90        11
          12       1.00      0.20      0.33         5
          13       0.89      0.95      0.92       187
          14       1.00      0.33      0.50         3
          15       0.92      0.94      0.93       143
          16       1.00      0.58      0.73        19
          17       0.97      1.00      0.99       690
          18       0.92    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
clf6.fit(X_train_tfidf,y_train_tfidf)
y_test_pred_tfidf = clf6.predict(X_test_tfidf)
print(classification_report(y_test_tfidf,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       181
           1       0.95      0.85      0.89       110
           2       0.93      0.97      0.95        38
           3       0.98      0.98      0.98       147
           4       1.00      0.95      0.97        20
           5       0.99      1.00      0.99      1207
           6       0.98      0.94      0.96       131
           8       0.99      0.99      0.99       259
           9       0.97      0.97      0.97        38
          10       1.00      0.83      0.91         6
          11       1.00      1.00      1.00        11
          12       0.50      0.20      0.29         5
          13       0.96      0.99      0.98       187
          14       0.50      0.67      0.57         3
          15       0.95      1.00      0.98       143
          16       1.00      0.68      0.81        19
          17       1.00      1.00      1.00       690
          18       0.98    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Deep Learning algorithms

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from nltk.tokenize import word_tokenize

In [ ]:
tkz = Tokenizer()
tkz.fit_on_texts(feature_text)

In [ ]:
vocab_length = len(tkz.word_index) + 1
embd_sent = tkz.texts_to_sequences(feature_text)

In [ ]:
vocab_length

19878

In [ ]:
longest_sent_len = max([ len(x.split()) for x in feature_text])

In [ ]:
pad_sent = pad_sequences(embd_sent, longest_sent_len, padding='post')

In [ ]:
embd_dict = dict()

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.50d.txt', encoding="utf8") as glove_file:
    for line in glove_file:
        line = glove_file.readline()
        records = line.split()
        word = records[0]
        vec = np.asarray(records[1:], dtype='float32')
        embd_dict[word] = vec


In [ ]:
len(embd_dict.keys())

200000

In [ ]:
embd_matrix = np.zeros((vocab_length, 50))
for word, index in tkz.word_index.items():
    embd_vec = embd_dict.get(word)
    if embd_vec is not None:
        embd_matrix[index] = embd_vec

In [ ]:
len(embd_matrix)

19878

In [ ]:
class_count = len(classes)
class_count

28

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 50, weights=[embd_matrix], input_length=longest_sent_len, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(class_count, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 516, 50)           993900    
_________________________________________________________________
flatten (Flatten)            (None, 25800)             0         
_________________________________________________________________
dense (Dense)                (None, 8)                 206408    
_________________________________________________________________
dense_1 (Dense)              (None, 28)                252       
Total params: 1,200,560
Trainable params: 206,660
Non-trainable params: 993,900
_________________________________________________________________
None


In [ ]:
dummy_label = np_utils.to_categorical(label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pad_sent, dummy_label, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
492/492 [==============================] - 5s 9ms/step - loss: 0.2497 - acc: 0.3896
Epoch 2/100
492/492 [==============================] - 4s 9ms/step - loss: 0.0637 - acc: 0.7185
Epoch 3/100
492/492 [==============================] - 4s 9ms/step - loss: 0.0474 - acc: 0.8054
Epoch 4/100
492/492 [==============================] - 4s 9ms/step - loss: 0.0373 - acc: 0.8525
Epoch 5/100
492/492 [==============================] - 4s 9ms/step - loss: 0.0306 - acc: 0.8738
Epoch 6/100
492/492 [==============================] - 4s 8ms/step - loss: 0.0253 - acc: 0.8960
Epoch 7/100
492/492 [==============================] - 4s 8ms/step - loss: 0.0231 - acc: 0.9065
Epoch 8/100
492/492 [==============================] - 4s 9ms/step - loss: 0.0208 - acc: 0.9161
Epoch 9/100
492/492 [==============================] - 4s 8ms/step - loss: 0.0188 - acc: 0.9239
Epoch 10/100
492/492 [==============================] - 4s 8ms/step - loss: 0.0177 - acc: 0.9285
Epoch 11/100
492/492 [=================

In [ ]:
model.save("main_cat_model.h5")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 84.057969


In [ ]:
print(y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
y_test_pred = model.predict_classes(X_test)
print(classification_report( np.argmax(y_test, axis=1),y_test_pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.85      0.88      0.87       181
           1       0.68      0.59      0.63       110
           2       0.61      0.71      0.66        38
           3       0.73      0.71      0.72       147
           4       0.82      0.70      0.76        20
           5       0.94      0.96      0.95      1207
           6       0.81      0.85      0.83       131
           8       0.85      0.74      0.79       259
           9       0.97      0.74      0.84        38
          10       0.29      0.33      0.31         6
          11       0.19      0.36      0.25        11
          12       0.00      0.00      0.00         5
          13       0.72      0.75      0.74       187
          14       0.00      0.00      0.00         3
          15       0.79      0.78      0.78       143
          16       0.24      0.21      0.22        19
          17       0.90      0.94      0.92       690
          18       0.70    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
